In [1]:
import pandas as pd
df_domain = pd.read_csv('ideo_domain_mbfc081123_with_weights.tsv', sep='\t')

/var/folders/8v/r0lk2mbn2k5dgm_pdmv4_8hr0000gn/T/ipykernel_71414/3110861098.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [43]:
from selenium import webdriver 
from selenium.webdriver.common.by import By

# Create Chromeoptions instance 
options = webdriver.ChromeOptions() 

# set header and headless mode
options.add_argument('user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36')
options.add_argument('--headless')  

# Adding argument to disable the AutomationControlled flag 
options.add_argument("--disable-blink-features=AutomationControlled") 
 
# Exclude the collection of enable-automation switches 
options.add_experimental_option("excludeSwitches", ["enable-automation"]) 
 
# Turn-off userAutomationExtension 
options.add_experimental_option("useAutomationExtension", False) 
 


In [57]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import StaleElementReferenceException, TimeoutException
import re

donation_keywords = [
    "donate", "donation"
]

def check_elements(elements):
    for element in elements:
        # Check if the element is visible and clickable
        if element.is_displayed() and element.is_enabled():
            # Check the element's text and attributes for donation-related keywords
            text = element.text.lower()
            href = element.get_attribute("href") or ""
            for keyword in donation_keywords:
                if keyword in text or keyword in href:
                    print(f"Donation option found: {text}; {href}")
                    return True
    return False
def find_donation_option(url):
    print(url)
    driver = webdriver.Chrome(options=options) 
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})") 
    
    try:
        driver.get('https://' + url)
        for keyword in donation_keywords:
            links = driver.find_elements(By.PARTIAL_LINK_TEXT, keyword)
            buttons = driver.find_elements(By.XPATH, f"//button[contains(translate(., 'ABCDEFGHIJKLMNOPQRSTUVWXYZ', 'abcdefghijklmnopqrstuvwxyz'), '{keyword}')]")
            if check_elements(links + buttons):
                return 1

            iframes = driver.find_elements(By.TAG_NAME, "iframe")
            for i in range(len(iframes)):
                iframes = driver.find_elements(By.TAG_NAME, "iframe")
                if i >= len(iframes):
                    break

                iframe = iframes[i]
                title = iframe.get_attribute("title") or ""
                if keyword in title.lower():
                    print(f"Donation option found in iframe title: {title}")
                    return 1

                driver.switch_to.frame(iframe)
                clickable_elements = driver.find_elements(By.XPATH, f"//*[contains(@href, '{keyword}') or contains(text(), '{keyword.capitalize()}')]")
                if check_elements(clickable_elements):
                    return 1
                driver.switch_to.default_content()

        print("Donation option not found")
        return 0

    except TimeoutException as e:
        print(f"TimeoutException encountered while processing {url}, skipping...")
        return 'failed'
    except Exception as e:
        print(f"Encountered an error of type {type(e).__name__} while processing {url}, skipping...")
        return 'failed'
    finally:
        driver.quit()

In [59]:
find_donation_option('castanet.net')

castanet.net
Donation option not found


0

In [60]:
start = 100
end = 1100
df_domain_partition = df_domain[start:end]
df_domain_partition['donation'] = df_domain_partition['domain'].apply(find_donation_option)
df_domain_partition.to_csv(f'donation_df/{start}_{end}.csv')


economictimes.indiatimes.com
Donation option not found
aleteia.org
Donation option not found
makeuseof.com
Donation option not found
livemint.com
Donation option found: electoral bonds: the curious case of madanlal ltd’s donations; https://www.livemint.com/politics/news/electoral-bonds-the-curious-case-of-madanlal-ltd-s-donations-11710684590889.html
ctvnews.ca
Donation option not found
scmp.com
Donation option not found
apnews.com
Donation option found: donate; 
polygon.com
Donation option not found
hollywoodreporter.com
Donation option not found
washingtonexaminer.com
Donation option not found
lapresse.ca
Donation option not found
thehindu.com
Donation option found: ncp received its largest single-day donation of electoral bonds while in power; https://www.thehindu.com/data/ncp-received-its-largest-single-day-donation-of-electoral-bonds-while-in-power/article67962173.ece
heavy.com
Donation option not found
dailywire.com
Donation option not found
elitedaily.com
Donation option not foun

/var/folders/8v/r0lk2mbn2k5dgm_pdmv4_8hr0000gn/T/ipykernel_71414/2106002205.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_domain_partition['donation'] = df_domain_partition['domain'].apply(find_donation_option)
